# Collect demonstrations

Interactive notebook for collecting demonstrations

In [1]:
%matplotlib auto

import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pathlib
import socket
from datetime import datetime
from demonstration_recorder import DemonstrationRecorder

# the gamepad is only available on some machines, and only on Linux
available_gamepad = True
try:
    from robotcontrol.gamepad_controller import GamepadController
except ModuleNotFoundError:
    print("Approxeng module not found, cannot use gamepad")
    available_gamepad = False
    
from robotcontrol.keyboard_controller import KeyboardController
from robotcontrol.automove_controller import AutoMoveController
from robot.al5d_position_controller import PositionController
from robot.al5d_simulated_position_controller import SimulatedPositionController


from camera.camera_controller import CameraController
from demonstration import Demonstration


Using matplotlib backend: module://matplotlib_inline.backend_inline


In [2]:
experiment = "demonstration_collector"
# picking up a collection specific name
hostname = socket.gethostname()
print(f"Hostname is {hostname}")
if hostname == "raven":
    run = "collect_freeform_raven"
elif hostname == "szenes.local":
    run = "collect_no_robot"
elif hostname == "tredy2":
    run = "tredy2_collect_freeform"
    #run = "tredy2_automove_rwp6d_00"
else:
    run = "collect_freeform"

exp = Config().get_experiment(experiment, run)
exp_target = Config().get_experiment("demonstration", exp["demonstration_name"])
exp_position_controller = Config().get_experiment(
    exp["exp_position_controller"], 
    exp["run_position_controller"])

exp_camera_controller = Config().get_experiment(exp["exp_camera_controller"], exp["run_camera_controller"])
exp_keyboard_controller = Config().get_experiment(exp["exp_keyboard_controller"], exp["run_keyboard_controller"])
exp_gamepad_controller = Config().get_experiment(exp["exp_gamepad_controller"], exp["run_gamepad_controller"])
exp_automove_controller = Config().get_experiment(exp["exp_automove_controller"], exp["run_automove_controller"])


Hostname is tredy2
***ExpRun**: Loading pointer config file:
	/home/lboloni/.config/BerryPicker/mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	/home/lboloni/Insync/lotzi.boloni@gmail.com/Google Drive/LotziStudy/Code/PackageTracking/BerryPicker/settings/settings-tredy2.yaml
***ExpRun**: Configuration for exp/run: demonstration_collector/tredy2_collect_freeform successfully loaded
***ExpRun**: Configuration for exp/run: demonstration/freeform successfully loaded
***ExpRun**: Configuration for exp/run: robot_al5d/position_controller_00 successfully loaded
***ExpRun**: Experiment default config /home/lboloni/Documents/Hackingwork/_Checkouts/BerryPicker/BerryPicker/src/experiment_configs/controllers/_defaults_controllers.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: controllers/camera_controller_tredy2 successfully loaded
***ExpRun**: Experiment default config /home/lboloni/Documents/Hackingwork/_Checkouts/BerryPicker/BerryPicker/src/experiment_configs/c

In [3]:
# This was here to debug things, because it was not finding one of the cameras
import cv2

# identifying which cameras work
for i in range(10):
    cap = cv2.VideoCapture(i)
    # FIXME: set for the specified one
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 256)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 256)
    cap.set(cv2.CAP_PROP_FPS, 10)

    ret, frame = cap.read()
    if ret:
        print(f"Camera {i} is working")
    else:
        print(f"Camera {i} not working")
    cap.release()

Camera 0 is working
Camera 1 is working
Camera 2 not working
Camera 3 not working
Camera 4 not working
Camera 5 not working
Camera 6 not working
Camera 7 not working
Camera 8 not working
Camera 9 not working


[ WARN:0@4.687] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video2): can't open camera by index
[ERROR:0@4.688] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@4.688] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video3): can't open camera by index
[ERROR:0@4.689] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@4.689] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video4): can't open camera by index
[ERROR:0@4.690] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@4.690] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video5): can't open camera by index
[ERROR:0@4.690] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@4.691] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video6): can't open camera by index
[ERROR:0@4.691] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup C

In [4]:
# starting the robot controller
if exp["available_robot"]:
    robot_controller = PositionController(exp_position_controller)
else:
    robot_controller = SimulatedPositionController(exp_position_controller)
# starting the camere controller
camera_controller = CameraController(exp_camera_controller)
camera_controller.visualize = True


task_dir = exp_target.data_dir()
demoname = datetime.now().strftime('%Y_%m_%d__%H_%M_%S')
demo_dir = pathlib.Path(task_dir, demoname)
demo_dir.mkdir(parents=False, exist_ok=False)
print(f"Ok, demonstration with go into\n\t {demo_dir}")

demonstration = Demonstration(exp_target, demoname, parse_old_style = False)
demonstration.metadata["cameras"] = sorted(camera_controller.capture_devs.keys())
demonstration.save_metadata()
# fixme need to add the cameras into the demonstration

***ExpRun**: Configuration for exp/run: robot_al5d/pulse_controller_00 successfully loaded
***ExpRun**: Configuration for exp/run: robot_al5d/angle_controller_00 successfully loaded
Try out the backup /dev/ttyUSB0
{'height': 5.0, 'distance': 5.0, 'heading': 0.0, 'wrist_angle': -45.0, 'wrist_rotation': 75.0, 'gripper': 100}
cap0 works
cap1 works
Ok, demonstration with go into
	 /home/lboloni/Documents/Hackingwork/__Temporary/BerryPicker-experiments/demonstration/freeform/2025_12_29__13_39_09
Saved demonstration metadata to
	/home/lboloni/Documents/Hackingwork/__Temporary/BerryPicker-experiments/demonstration/freeform/2025_12_29__13_39_09/_metadata.yaml


In [5]:
# Performs the recording. The robot is controlled through the primary 
# controller. This is also used to terminate the collection
# The information is collected by the demonstration recorder

controller = exp["primary_controller"]

if controller == "xbox":
    gamepad_controller = GamepadController(exp_gamepad_controller,
        robot_controller=robot_controller, camera_controller=camera_controller)
    demo_recorder = DemonstrationRecorder(demonstration,
        camera_controller=camera_controller, remote_control=gamepad_controller, robot_controller=
                                robot_controller, save_dir=demo_dir)
    # dr = None
    gamepad_controller.demonstration_recorder = demo_recorder
    gamepad_controller.control()
    demonstration.save_metadata()
    print("====== Demonstration terminated and recorded successfully, bye. ======")
if controller == "keyboard":
    kb_controller = KeyboardController(exp_keyboard_controller,
        robot_controller=robot_controller, camera_controller=camera_controller)
    demo_recorder = DemonstrationRecorder(demonstration, camera_controller=camera_controller, remote_control=kb_controller, robot_controller=
                                robot_controller, save_dir=demo_dir)
    # dr = None
    kb_controller.demonstration_recorder = demo_recorder
    kb_controller.control()
    demonstration.save_metadata()
    print("====== Demonstration terminated and recorded successfully, bye. ======")

if controller == "automove":
    automove_controller = AutoMoveController(exp_automove_controller,
        robot_controller=robot_controller, camera_controller=camera_controller)
    automove_controller.generate_waypoints()
    # program_controller.interactive_confirm = False
    demo_recorder = DemonstrationRecorder(demonstration, camera_controller=camera_controller, remote_control=automove_controller, robot_controller=
                                robot_controller, save_dir=demo_dir)
    automove_controller.demonstration_recorder = demo_recorder
    demonstration.save_metadata()
    automove_controller.control()


 height:5.00
 distance:5.00
 heading:0.00
 wrist_angle:-45.00
 wrist_rotation:75.00
 gripper:100.00



Found a joystick and connected
{'axes': ['l', 'lt', 'lx', 'ly', 'r', 'rt', 'rx', 'ry'], 'buttons': ['circle', 'cross', 'ddown', 'dleft', 'dright', 'dup', 'home', 'l1', 'ls', 'r1', 'rs', 'select', 'square', 'start', 'triangle']}


 height:5.00
 distance:5.00
 heading:0.00
 wrist_angle:-45.00
 wrist_rotation:75.00
 gripper:100.00

 height:5.00
 distance:5.00
 heading:0.00
 wrist_angle:-45.00
 wrist_rotation:75.00
 gripper:100.00

 height:5.00
 distance:5.00
 heading:0.00
 wrist_angle:-45.00
 wrist_rotation:75.00
 gripper:100.00

 height:5.00
 distance:5.00
 heading:0.00
 wrist_angle:-45.00
 wrist_rotation:75.00
 gripper:100.00

 height:5.00
 distance:5.00
 heading:0.00
 wrist_angle:-45.00
 wrist_rotation:75.00
 gripper:100.00

 height:5.00
 distance:5.00
 heading:0.00
 wrist_angle:-45.00
 wrist_rotation:75.00
 gripper:100.00

 height:5.00
 distance:5.00
 heading:0.00
 wrist_angle:-45.00
 wrist_rotation:75.00
 gripper:100.00

 height:5.00
 distance:5.00
 heading:0.00
 wrist_angle:-45.00
 wrist_rotation:75.00
 gripper:100.00

 height:5.00
 distance:5.00
 heading:0.00
 wrist_angle:-45.00
 wrist_rotation:75.00
 gripper:100.00

 height:5.00
 distance:5.00
 heading:0.00
 wrist_angle:-45.00
 wrist_rotation:75.00
 grippe

['square']
***al5d_position_controller: Initiating the stopping of the robot
***al5d_position_controller: Robot stopped
Saved demonstration metadata to
	/home/lboloni/Documents/Hackingwork/__Temporary/BerryPicker-experiments/demonstration/freeform/2025_12_29__13_39_09/_metadata.yaml
*** Gamepack Controller: Exiting, stopping the robot.
***al5d_position_controller: Initiating the stopping of the robot
***al5d_position_controller: Robot stopped
*** Gamepack Controller: Robot stopped, leaving GamepadController
Saved demonstration metadata to
	/home/lboloni/Documents/Hackingwork/__Temporary/BerryPicker-experiments/demonstration/freeform/2025_12_29__13_39_09/_metadata.yaml
====== Demonstration terminated and recorded successfully, bye. ======


In [6]:
demonstration.save_metadata()
demonstration.move_to_video(delete_img_files=True)


Saved demonstration metadata to
	/home/lboloni/Documents/Hackingwork/__Temporary/BerryPicker-experiments/demonstration/freeform/2025_12_29__13_39_09/_metadata.yaml
***Demonstration***: moving to video started
move to video per camera dev0
move to video per camera dev1
Saved demonstration metadata to
	/home/lboloni/Documents/Hackingwork/__Temporary/BerryPicker-experiments/demonstration/freeform/2025_12_29__13_39_09/_metadata.yaml
***Demonstration***: moving to video done


In [7]:
print(demonstration.metadata)

{'stored_as_video': True, 'stored_as_images': False, 'maxsteps': 118, 'cameras': ['dev0', 'dev1']}


In [8]:
exp

Experiment:
    available_robot: true
    data_dir: /home/lboloni/Documents/Hackingwork/__Temporary/BerryPicker-experiments/demonstration_collector/tredy2_collect_freeform
    demonstration_name: freeform
    exp_automove_controller: automove
    exp_camera_controller: controllers
    exp_gamepad_controller: controllers
    exp_keyboard_controller: controllers
    exp_position_controller: robot_al5d
    exp_run_sys_indep_file: /home/lboloni/Documents/Hackingwork/_Checkouts/BerryPicker/BerryPicker/src/experiment_configs/demonstration_collector/tredy2_collect_freeform.yaml
    experiment_name: demonstration_collector
    primary_controller: xbox
    run_automove_controller: automove_random_waypoint_6D_00
    run_camera_controller: camera_controller_tredy2
    run_gamepad_controller: gamepad_x360_controller
    run_keyboard_controller: keyboard_controller
    run_name: tredy2_collect_freeform
    run_position_controller: position_controller_00
    subrun_name: null